# Threads
***

Se divide em dois modulos: **_thread** e **threading**

**threads** são como forks entretanto são usados para executar objetos em um mesmo processo, o que lhes garante uma melhor performace, simplicidade, compartilhamento de memória e portabilidade.

Um dos problemas mais notáveis de threads é para sincronizar operações, uma vez que mesmo operações simples como printar texto na tela pode gerar conflitos. Veremos a frente que por debaixo dos panos de fato apenas uma thread está sendo realmente executada pelo interpretador de python.

Um cuidado que devemos ter com threads é ao compartilhar objetos, pois se duas threads tentarem modificar o objeto ao mesmo tempo isso poderá causar erros terríveis. Para compartilhar objetos o melhor é travar cada um deles.

**Módulo _thread**: Realiza operações de baixo nível.

**Módulo threading**: Implementa objetos baseados no módulo _thread, logo utilizar essa.

In [1]:
import math
from time import time
import threading


def computer(start: int, end: int) -> None:
    """
    Realiza o pode computacional.
    """

    print(f"Iniciando o cálculo {threading.current_thread().name}\n")

    i = start
    factor = 1000 * 1000
    while i < end:
        i += 1
        math.sqrt((i - factor) * (i - factor))
        
    print(f"Finalizando o cálculo {threading.current_thread().name}\n")

In [2]:
start_time = time()
computer(1, 50_000_000)
end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Iniciando o cálculo MainThread

Finalizando o cálculo MainThread

Requisição executada em 21.9 segundos


***
### Thread Simples
***

In [3]:
from concurrent.futures.thread import ThreadPoolExecutor

start_time = time()
with ThreadPoolExecutor() as executor:
    executor.submit(computer, start=1, end=50_000_000)

end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Iniciando o cálculo ThreadPoolExecutor-0_0

Finalizando o cálculo ThreadPoolExecutor-0_0

Requisição executada em 21.51 segundos


***
### Multiplas Threads
***

Mesmo usando multiplas threads o GIL do python bloqueia elas e faz com que rodem de forma sincrona

In [4]:
from concurrent.futures.thread import ThreadPoolExecutor

In [5]:
start_time = time()
with ThreadPoolExecutor(max_workers=8) as executor:
    for n in range(1, 9):
        initial = 50_000_000 * (n - 1) / 8
        end = 50_000_000 * n / 8
        print(f"Thread {n} processando de {initial} até {end}.")
        executor.submit(computer, start=initial, end=end)

end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Thread 1 processando de 0.0 até 6250000.0.
Iniciando o cálculo ThreadPoolExecutor-1_0

Thread 2 processando de 6250000.0 até 12500000.0.
Iniciando o cálculo ThreadPoolExecutor-1_1

Thread 3 processando de 12500000.0 até 18750000.0.
Iniciando o cálculo ThreadPoolExecutor-1_2

Thread 4 processando de 18750000.0 até 25000000.0.
Iniciando o cálculo ThreadPoolExecutor-1_3

Thread 5 processando de 25000000.0 até 31250000.0.
Iniciando o cálculo ThreadPoolExecutor-1_4

Thread 6 processando de 31250000.0 até 37500000.0.
Iniciando o cálculo ThreadPoolExecutor-1_5

Thread 7 processando de 37500000.0 até 43750000.0.
Iniciando o cálculo ThreadPoolExecutor-1_6

Thread 8 processando de 43750000.0 até 50000000.0.
Iniciando o cálculo ThreadPoolExecutor-1_7

Finalizando o cálculo ThreadPoolExecutor-1_4

Finalizando o cálculo ThreadPoolExecutor-1_7

Finalizando o cálculo ThreadPoolExecutor-1_0

Finalizando o cálculo ThreadPoolExecutor-1_2

Finalizando o cálculo ThreadPoolExecutor-1_5

Finalizando o cálcu

***
### Multiplas thread em background
***

In [6]:
from time import time, sleep
import threading

In [7]:
def countdown(count: int) -> None:
    """
    Contando para baixo.
    """

    while count >= 0:
        print(f"Contagem regressiva na thread {threading.current_thread().name}: {count}")
        count -= 1
        sleep(3)

In [8]:
def countup(count: int) -> None:
    """
    Contando para cima.
    """

    while count <= 10:
        print(f"Contagem progressiva na thread {threading.current_thread().name}: {count}")
        count += 1
        sleep(5)

In [9]:
start_time = time()
print("Criando a threads e inserindo-as na pool de threads prontas para execução do processador.")
t1 = threading.Thread(name="countdown", args=(10,), target=countdown)
t1.start()

t2 = threading.Thread(name="countup", args=(0,), target=countup)
t2.start()

print("Threads inseridas no pool!")

end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Criando a threads e inserindo-as na pool de threads prontas para execução do processador.
Contagem regressiva na thread countdown: 10
Contagem progressiva na thread countup: 0
Threads inseridas no pool!
Requisição executada em 0.0 segundos
Contagem regressiva na thread countdown: 9


***
### Thread com Queue
***

In [10]:
import threading
import queue
from time import time

Contagem progressiva na thread countup: 1
Contagem regressiva na thread countdown: 8
Contagem regressiva na thread countdown: 7
Contagem progressiva na thread countup: 2
Contagem regressiva na thread countdown: 6
Contagem regressiva na thread countdown: 5
Contagem progressiva na thread countup: 3
Contagem regressiva na thread countdown: 4
Contagem progressiva na thread countup: 4
Contagem regressiva na thread countdown: 3
Contagem regressiva na thread countdown: 2
Contagem progressiva na thread countup: 5
Contagem regressiva na thread countdown: 1
Contagem progressiva na thread countup: 6
Contagem regressiva na thread countdown: 0
Contagem progressiva na thread countup: 7
Contagem progressiva na thread countup: 8
Contagem progressiva na thread countup: 9
Contagem progressiva na thread countup: 10


In [11]:
def generate_data(queue: queue.Queue) -> None:
    """
    Gera os dados e insere na queue.
    """

    print(f"Gerando dados na {threading.current_thread().name}")

    for i in range(1, 11):
        print(f"Dado {i} gerado.")
        queue.put(i)

In [12]:
def process_data(queue: queue.Queue) -> None:
    """
    Processa os dados recebidos.
    """
    print(f"Processando dados na {threading.current_thread().name}")

    while queue.qsize() > 0:
        value = queue.get()
        print(f"Dado {value * 2} processado.")
        queue.task_done()

In [13]:
start_time = time()
data_queue = queue.Queue()
print(f"Iniciando a thread {threading.current_thread().name}")

thread_generator = threading.Thread(target=generate_data, args=(data_queue,))
thread_processor = threading.Thread(target=process_data, args=(data_queue,))

thread_generator.start()
thread_generator.join()

thread_processor.start()
thread_processor.join()

end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Iniciando a thread MainThread
Gerando dados na Thread-5 (generate_data)
Dado 1 gerado.
Dado 2 gerado.
Dado 3 gerado.
Dado 4 gerado.
Dado 5 gerado.
Dado 6 gerado.
Dado 7 gerado.
Dado 8 gerado.
Dado 9 gerado.
Dado 10 gerado.
Processando dados na Thread-6 (process_data)
Dado 2 processado.
Dado 4 processado.
Dado 6 processado.
Dado 8 processado.
Dado 10 processado.
Dado 12 processado.
Dado 14 processado.
Dado 16 processado.
Dado 18 processado.
Dado 20 processado.
Requisição executada em 0.0 segundos


***
### Threads com lock
***

In [14]:
import threading
import random
from time import time

In [15]:
class BankAccount:
    """
    Conta bancária.
    """

    def __init__(self, saldo=0) -> None:
        """
        Construtor.
        """

        self.saldo = saldo

    def transfer(self, destination_account: "BankAccount", value: int) -> None:
        """
        Realiza a transferência entre as contas.
        """

        if self.saldo < value:
            return

        self.saldo -= value
        destination_account.saldo += value

In [16]:
lock = threading.RLock()
qtd_inconsistent = 0

In [17]:
def create_bank_accounts() -> list[BankAccount]:
    """
    Cria 5 contas com saldo entre 5 mil e 10 mil reais.
    """

    return [
        BankAccount(saldo=random.randint(5_000, 10_000)),
        BankAccount(saldo=random.randint(5_000, 10_000)),
        BankAccount(saldo=random.randint(5_000, 10_000)),
        BankAccount(saldo=random.randint(5_000, 10_000)),
        BankAccount(saldo=random.randint(5_000, 10_000))
    ]

In [18]:
def get_random_accounts(accounts: list[BankAccount]) -> tuple[BankAccount, BankAccount]:
    """
    Pega 2 contas randomicamente para realizar as transferências.
    """

    origin_account = random.choice(accounts)
    destination_account = random.choice(accounts)

    while origin_account == destination_account:
        destination_account = random.choice(accounts)

    return origin_account, destination_account

In [19]:
def verify_bank_integrity(accounts: list[BankAccount], total: int) -> None:
    """
    Valida a integridade dos dados do banco se o saldo permanece o mesmo após as transferências.
    """

    with lock:
        current = sum(account.saldo for account in accounts)

    if current != total:
        print(f"ERRO: Balanço bancário inconsistente. Atual R$ {current:.2f}, Total R$ {total:.2f}")
        with lock:
            qtd_inconsistent += 1
    else:
        print(f"SUCESSO: Balanço bancário consistente. Atual R$ {current:.2f}, Total R$ {total:.2f}")

In [20]:
def services(accounts: list[BankAccount], total: int) -> None:
    """
    Realiza as transferências.
    """

    print(f"Iniciando services na thread {threading.current_thread().name}")

    for _ in range(1, 2_000):
        origin_account, destination_account = get_random_accounts(accounts)
        value = random.randint(1, 100)
        with lock:
            origin_account.transfer(destination_account, value)

        verify_bank_integrity(accounts, total)

In [21]:
start_time = time()
print(f"Iniciando a thread {threading.current_thread().name}")

bank_accounts = create_bank_accounts()
with lock:
    total = sum(account.saldo for account in bank_accounts)

print(f"O saldo total das contas é de {total:.2f}")
print("Iniciando transferências...")

tasks = [
    threading.Thread(target=services, args=(bank_accounts, total)),
    threading.Thread(target=services, args=(bank_accounts, total)),
    threading.Thread(target=services, args=(bank_accounts, total)),
    threading.Thread(target=services, args=(bank_accounts, total)),
    threading.Thread(target=services, args=(bank_accounts, total))
]
[task.start() for task in tasks]
[task.join() for task in tasks]

print(f"Transferências completadas com sucesso com {qtd_inconsistent} inconsistências.")
verify_bank_integrity(bank_accounts, total)

end_time = time() - start_time
print(f"Requisição executada em {round(end_time, 2)} segundos")

Iniciando a thread MainThread
O saldo total das contas é de 36471.00
Iniciando transferências...
Iniciando services na thread Thread-7 (services)
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço bancário consistente. Atual R$ 36471.00, Total R$ 36471.00
SUCESSO: Balanço b